# Preparation 

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
import langchain as lc

import warnings
warnings.filterwarnings('ignore')

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
#from langchain.prompts.prompt import PromptTemplate

llm = lc.OpenAI(temperature=0)

_PROMPT_TEMPLATE = """You are an expert professor specialized in grading students' answers to questions.
You are grading the following question:
{query}
Here is the real answer:
{answer}
You are grading the following predicted answer:
{result}
What grade do you give from 0 to 10, where 0 is the lowest (very low similarity) and 10 is the highest (very high similarity)?
"""

PROMPT = lc.PromptTemplate(
    input_variables=["query", "answer", "result"], 
    template=_PROMPT_TEMPLATE
)

In [3]:
context_examples = [
    {
        "question": "Do you offer vegetarian or vegan options?",
        "context": "Yes, we have a range of dishes to cater to vegetarians and vegans",
    },
    {
        "question": "What are the hours of operation for your restaurant?",
        "context": "Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday. On Sundays, we open at 12 p.m. and close at 9 p.m.",
    },
]
QA_PROMPT = "Answer the question based on the  context\nContext:{context}\nQuestion:{question}\nAnswer:"
template = lc.PromptTemplate(input_variables=["context", "question"], 
                             template=QA_PROMPT)
qa_chain = lc.chains.LLMChain(llm=llm, prompt=template)
predictions = qa_chain.apply(context_examples)
predictions

[{'text': 'Yes, we offer vegetarian and vegan options.'},
 {'text': 'Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday, and from 12 p.m. to 9 p.m. on Sundays.'}]

In [9]:
eval_chain = lc.evaluation.qa.ContextQAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(
    context_examples, predictions, question_key="question", prediction_key="text"
)
graded_outputs

[{'text': ' CORRECT'}, {'text': ' CORRECT'}]